In [1]:
import re

In [2]:
datafile = "../result.txt"

destination_file = 'result.txt'

In [3]:
with open(datafile) as f:
    data = f.readlines()

In [4]:
def expect_that(output, expect):
    assert output == expect, "Output:\n{}\nExpect:\n{}".format(output, expect)
    

def get_problem_search_id(sent):
    '''Returns (problem, search) or False
    '''
    pattern = re.compile(r'Problem: (\d+) Search: (\d+)')
    found = pattern.search(sent)
    if found:
        return found.groups()
    return False

def get_search_type(sent):
    '''Returns [(search_type)] or []
    '''
    pattern = re.compile('using\s([A-z_\s\d]+)')
    found = pattern.search(sent)
    if found:
        return found[1]
    return False

def data_check(sent):
    if "Expansions   Goal Tests   New Nodes" in sent:
        return True
    return False

def get_data(sent):
    num_list = sent.strip().split(' ')
    num_list = [digit for digit in  num_list if len(digit) > 0]
    assert len(num_list) == 3, "num_list: {}".format(num_list)
    
    result = {
        'expansions': int(num_list[0]),
        'goal_tests': int(num_list[1]),
        'new_nodes': int(num_list[2]),
    }
    
    return result

def get_plan_second(sent):
    data = sent.strip().split(' ')
    return (int(data[2]), float(data[-1]))

In [5]:
test_input = "Problem: 1 Search: 1\n"
expect = ('1', '1')
expect_that(get_problem_search_id(test_input), expect)

test_input = "Solving Air Cargo Problem 1 using breadth_first_search...\n"
expect = "breadth_first_search"
expect_that(get_search_type(test_input), expect)

test_input = "Expansions   Goal Tests   New Nodes"
expect = True
expect_that(data_check(test_input), expect)

test_input =  '    43          56         180    \n'
expect = {'expansions': 43, 'goal_tests': 56, 'new_nodes': 180}
expect_that(get_data(test_input), expect)

test_input = 'Plan length: 6  Time elapsed in seconds: 0.031694407007307746\n'
expect = (6, 0.031694407007307746)
expect_that(get_plan_second(test_input), expect)

In [6]:
line_begin = False
line_end = False

with open(destination_file, 'w') as f:
    for i in range(len(data)):
        line = data[i]

        problem_check = get_problem_search_id(line)
        search_check = get_search_type(line)
        is_next_line_data = False    

        if problem_check:
            if line_end is False and i > 0:
                print(",,,,", file=f)
            print("{},{},".format(problem_check[0], problem_check[1]), end='', file=f)
            line_begin = True
            line_end = False

        if search_check:
            print("{},".format(search_check), end='', file=f)        

        is_next_line_data = data_check(line)

        if is_next_line_data:
            next_line = data[i + 1]
            result_dict = get_data(next_line)
            print("{},{},{},".format(result_dict['expansions'], result_dict['goal_tests'], result_dict['new_nodes']), end='', file=f)       

            is_plan_second_line = get_plan_second(data[i + 3])
            print("{},{}".format(is_plan_second_line[0], is_plan_second_line[1]), file=f)
            line_end=True

        if i == len(data) - 1 and line_end is False:
            print(",,,,", file=f)
            
print("File Written to {}".format(destination_file))

File Written to result.txt
